In [1]:
import pandas as pd
import numpy as np
import datetime

# "Hea õppimise harjumuse" ülesande vastused
vastused = pd.read_csv("hea-õppimise-harjumus.csv", encoding='utf-8')
vastused = vastused[["Ajatempel", "Ees- ja perekonnanimi", "Kas sa oled täna õppinud (sh koduseid ülesandeid teinud)?"]]
vastused["Ajatempel"] = pd.to_datetime(vastused["Ajatempel"])
vastused['Ees- ja perekonnanimi'] = vastused['Ees- ja perekonnanimi'].str.strip()
vastused['Ees- ja perekonnanimi'] = vastused['Ees- ja perekonnanimi'].str.title()

# Link tudengite andmete juurde (Moodlest andmed)
tudengid = pd.read_csv("tudengid_moodlest.csv", encoding='utf-8')
tudengid = tudengid[["Eesnimi", "Perenimi", "Meiliaadress"]]
tudengid["Ees- ja perekonnanimi"] = tudengid.apply(lambda row: row["Eesnimi"] + " " + row["Perenimi"], axis=1)
tudengid['Ees- ja perekonnanimi'] = tudengid['Ees- ja perekonnanimi'].str.strip()

In [2]:
uhendatud = pd.merge(vastused, tudengid, on="Ees- ja perekonnanimi", how="left")

korrasRead = uhendatud[uhendatud['Meiliaadress'].notnull()]
korrasRead = korrasRead.sort_values(by=['Meiliaadress', 'Ajatempel'], ignore_index = True)

# Kogume kokku tudengite tulemused
tulemused = []

i = 0
# Käime läbi kõik tudengid
while i < len(korrasRead):
  
    # Tudengi isikukoodi leidmine
    meiliaadress = korrasRead['Meiliaadress'][i]
    tudeng = korrasRead[korrasRead["Meiliaadress"] == meiliaadress]
    nimi = tudeng["Ees- ja perekonnanimi"][i]

    # Ülesande alguskuupäev ning ühe päeva kestmine (ehk lisame alguskuupäevale 1 päeva juurde)
    algus = datetime.datetime(2022, 10, 25, 4, 0, 0).strftime('%Y/%m/%d %I:%M:%S')
    lopp = (pd.to_datetime(algus) + pd.DateOffset(days=1)).strftime('%Y/%m/%d %I:%M:%S')

    # Sissekannete arv
    plaanideArv = 0
    reflektsioonideArv = 0

    # Kokku oli 13 päeva, kontrollime kõikide päevade kohta sisestuste arvu
    j = 0
    for j in range(13):

        plaan = tudeng[
            (tudeng["Kas sa oled täna õppinud (sh koduseid ülesandeid teinud)?"] == "Ei (liigun edasi tegema plaani)") &
            (tudeng["Ajatempel"] > algus) &
            (tudeng["Ajatempel"] < lopp)]
        reflektsioon = tudeng[
            (tudeng["Kas sa oled täna õppinud (sh koduseid ülesandeid teinud)?"] == "Jah (liigun edasi tegema refleksiooni)") &
            (tudeng["Ajatempel"] > algus) &
            (tudeng["Ajatempel"] < lopp)]

        if plaan.shape[0] != 0:
            plaanideArv += 1
        if reflektsioon.shape[0] != 0:
            reflektsioonideArv += 1

        algus = lopp
        lopp = (pd.to_datetime(algus) + pd.DateOffset(days=1)).strftime('%Y/%m/%d %I:%M:%S')
        j += 1


    sissekanneteArv = plaanideArv + reflektsioonideArv
    tulemus = []
    tulemus.append(meiliaadress)
    tulemus.append(nimi)
    tulemus.append(plaanideArv)
    tulemus.append(reflektsioonideArv)
    tulemus.append(sissekanneteArv)
    if sissekanneteArv >= 20 :
        tulemus.append(8)
    elif sissekanneteArv >= 15 :
        tulemus.append(6)
    elif sissekanneteArv >= 10 :
        tulemus.append(4)
    elif sissekanneteArv >= 7 :
        tulemus.append(2)
    else:
        tulemus.append(0)

    tulemused.append(tulemus)
  
    i += len(tudeng)
  

In [3]:
# vigasedRead - seda saab kasutada, et näha, millised tudengid kas kirjutasid oma nime valesti või on mõni muu viga sees
# Nende ridade punktid tuleks tulemusesse ise juurde lisada.
vigasedRead = uhendatud[pd.isnull(uhendatud['Meiliaadress'])]
vigasedRead = vigasedRead.sort_values(by=['Ees- ja perekonnanimi', 'Ajatempel'], ignore_index = True)

In [4]:
# Andmete eksportimine faili
andmed_faili = pd.DataFrame(tulemused, columns = ["meiliaadress", "nimi", "plaanide arv", "reflektsioonide arv", "kokku", "punktid"])
eksport_faili_nimi = 'hea_oppimise_harjumuse_punktid.csv'
andmed_faili.to_csv(eksport_faili_nimi, sep=',', encoding='utf-8', index = False, columns = ["meiliaadress", "nimi", "plaanide arv", "reflektsioonide arv", "kokku", "punktid"]) 